# Silver Layer

Retirado do da branch da Mariana.


## Configuração do servidor para trabalho no Silver Layer

**Configuração do ambiente Spark**

In [0]:
# Importação das bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import gc

spark = (
    SparkSession.builder
        .appName("Silver Layer - Transformations")
        .config("spark.sql.shuffle.partitions", "200")
        .config("spark.sql.files.maxPartitionBytes", "134217728")  # 128MB
        .config("spark.sql.parquet.compression.codec", "snappy")
        .config("spark.sql.adaptive.enabled", "true")
        .config("spark.databricks.delta.optimizeWrite.enabled", "true")
        .config("spark.databricks.delta.autoCompact.enabled", "true")
        .getOrCreate()
)

# Conta quantos volumes já existem dentro do schema
numero = spark.sql("show volumes in data_ex.lhdw").count()
# Armazena o nome do último volume salvo
volume_salvo = f"download{numero:03d}"

# Definição dos path bronze e silver
bronze_path = f"/Volumes/data_ex/bronze/{volume_salvo}"
silver_path = f"/Volumes/data_ex/silver/{volume_salvo}"

spark.sql(f"create volume if not exists data_ex.silver.{volume_salvo}")

**Controle de versionamento por volume**

In [0]:
schema_name = "data_ex.lhdw"

numero_volumes = spark.sql(f"SHOW VOLUMES IN {schema_name}").count()
volume_atual = f"download{numero_volumes:03d}"

bronze_path = f"/Volumes/data_ex/bronze/{volume_atual}"
silver_path = f"/Volumes/data_ex/silver/{volume_atual}"

spark.sql(f"CREATE VOLUME IF NOT EXISTS data_ex.silver.{volume_atual}")

## Trabalhos no Silver Layer

#Logs


In [0]:
log_data{}

def make_log_tabel(camada):
    relacoes = {
            "bronze" : "bronze_log_carga",
            "silver" : "silver_log_carga",
            "gold"   : "gold_log_carga"
        }
    n = 0
    # Conta quantas tabelas existem
    try:
        n = spark.sql("show tables in data_ex.bronze").count()
    except:
        print("ERRO ao ler a tabela de logs")
    # Cria a tabela se não existir 
    if n == 0:
        try :
            spark.sql(f"""create table if not exists data_ex.bronze.{relacoes[camada]} (
                id_carga string,
                id_job string,
                nome_arquivo string,
                fonte string,
                camada string,
                path_origem string,
                path_destino string,
                data_inicio timestamp,
                data_fim timestamp,
                duracao_ms string,
                registros_lidos integer,
                registros_gravados integer,
                status string,
                mensagem_erro string,
                data_execusao date
                ) using delta """)
            return 1
        except Exception as e:
            print(f"Erro ao criar a tabela de logs: {e}")
            return 0
    # Erro, existe mais de uma tabela
    elif n > 1:
        print(f"Erro na árvore de diretórios, exite mais de {n} tabelas de losgs na camada bronze")
        return 0
    # A tabela ja existe
    else:
        print("Tabela de logs ja existes na camada bronze")
        return 1

def generate_log(volume, camada ,camada_destino, file, new_name):
    camada = camada.lower()
    id_job = {
        "bronze" : "meta_bronze",
        "silver" : "meta_silver",
        "gold"   : "meta_silver"
    }
  try:
    global log_data
    log_data.update({
      "id_carga" : str(uuid.uuid4()),
      "id_job"   : spark.sql(f"select * from data_ex.metadados.{id_job[camada]}").collect()[0][0],
      "nome_arquivo" : new_name,
      "fonte" : "filesystem_local",
      "camada" : camada,
      "path_origem" : f"/Volumes/data_ex/{camada_origem}/{volume}/{file}",
      "path_destino" : f"/Volumes/data_ex/{camada_destino}/{volume}/{new_name}",
      "data_inicio" : spark.range(1).select(current_timestamp()).collect()[0][0],
      "status" : "Running",
      "data_execusao" : spark.range(1).select(current_date()).collect()[0][0]
    })
    return 1
  except Exception as e:
    return 0

def update_log():
    try:
        global log_data
        log_data["status"] = "True"   # Atualiza o status do log
        data_fim = spark.range(1).select(current_timestamp()).collect()[0][0]
        log_data["data_fim"] = data_fim  
        duracao_ms = (data_fim - log_data[7]).total_seconds() * 1000
        log_data["duracao_ms"] = duracao_ms
        acao = storeLog(log_data)
        if acao == 1:
            return 1
        else:
            return 0 
    except Exception as e:
        print(f"Erro na funcao gerarLog(): {e}")

def storeLog():
    try:
        global log_data
        row = Row(
            id_carga = log_data["id_carga"],
            id_job = log_data["id_job"],
            nome_arquivo = log_data["nome_arquivo"],
            fonte = log_data["fonte"],
            camada = log_data["camada"],
            path_origem = log_data["path_origem"],
            path_destino = log_data["path_destino"],
            data_inicio = log_data["data_inicio"],
            data_fim = log_data["data_fim"],
            duracao_ms = log_data["duracao_ms"],
            registros_lidos = log_data["registros_lidos"],
            registros_gravados = log_data["registros_gravados"],
            status = log_data["status"],
            mensagem_erro = log_data["mensagem_erro"],
            data_carga = log_data["data_carga"]
        )

        df_log = spark.createDataFrame([row])
        df_log.write.mode("append").insertInto("data_ex.bronze.bronze_log_carga")
        return 1
    except Exception as e:
        print(f"Erro ao gravar o log: {e}")
        return 0

**Leitura das tabelas bronze**

In [0]:
bronze_tables = {
    "dim_categoria": "bronze_dim_categoria_produto",
    "dim_cliente": "bronze_dim_cliente",        
    "dim_data": "bronze_dim_data",
    "dim_localidade": "bronze_dim_localidade",                
    "dim_produto": "bronze_dim_produto",                    
    "fato_vendas": "bronze_fato_vendas"                                 
}

bronze_dfs = {                            
    nome: spark.read.format("delta").load(f"{bronze_path}/{tabela}")
    for nome, tabela in bronze_tables.items()
}

**Funções para a transformação**

In [0]:
# Função que substitui valores null por N/A
def replace_null_string(df):
    for coluna, tipo in df.dtypes:
        if tipo == "string" and not coluna.endswith("_id"):
            df = df.withColumn(                                            
                coluna,
                when(col(coluna).isNull(), lit("N/A")).otherwise(col(coluna))
            )
    return df

# Função para adicionar SKs e FKs
def add_surrogate_key(df, sk_name):
    window = Window.orderBy(lit(1))
    return df.withColumn(sk_name, row_number().over(window).cast("long"))

**Transformando dimensões**

In [0]:
def silver_categoria:
    # Dimensão Categoria
    silver_dim_categoria = (
        bronze_dfs["dim_categoria"]
            .dropDuplicates(["categoria_id"])
            .withColumn("categoria_nome", upper(trim(col("categoria_nome"))))
    )
    silver_dim_categoria = replace_null_string(silver_dim_categoria)
    silver_dim_categoria = add_surrogate_key(silver_dim_categoria, "sk_categoria")

def silver_cliente:
    # Dimensão Cliente
    silver_dim_cliente = (
        bronze_dfs["dim_cliente"]
            .dropDuplicates(["cliente_id"])
            .withColumn("nome_cliente", initcap(trim(col("nome_cliente"))))
            .withColumn("estado_cliente", upper(col("estado"))) #renomear coluna
            .withColumn("cidade_cliente", initcap(col("cidade"))) #renomear coluna
            .drop("estado", "cidade")
    )
    silver_dim_cliente = replace_null_string(silver_dim_cliente)
    silver_dim_cliente = add_surrogate_key(silver_dim_cliente, "sk_cliente")

def silver_data:
    # Dimensão Data
    silver_dim_data = (
        bronze_dfs["dim_data"]
            .dropDuplicates(["data_id"])
    )
    silver_dim_data = replace_null_string(silver_dim_data)
    silver_dim_data = add_surrogate_key(silver_dim_data, "sk_data")

def silver_localidade
# Dimensão Localidade
    silver_dim_localidade = (
        bronze_dfs["dim_localidade"]
            .dropDuplicates(["localidade_id"])
            .withColumn("estado_venda", upper(col("estado"))) # renomear coluna
            .withColumn("cidade_venda", initcap(col("cidade"))) # renomear coluna
            .drop("estado", "cidade")
    )
    silver_dim_localidade = replace_null_string(silver_dim_localidade)
    silver_dim_localidade = add_surrogate_key(silver_dim_localidade, "sk_localidade")

def silver_produto:
# Dimensão Produto
    silver_dim_produto = (
        bronze_dfs["dim_produto"]
            .dropDuplicates(["produto_id"])
            .drop("categoria_nome") # remover categoria_nome
            # .withColumn("categoria_nome", upper(col("categoria_nome")))
    )
    silver_dim_produto = replace_null_string(silver_dim_produto)
    silver_dim_produto = add_surrogate_key(silver_dim_produto, "sk_produto")

**Transformando a tabela fato**

In [0]:
def silver_fato_vendas:
    silver_fato_vendas = (
        bronze_dfs["fato_vendas"]
            .dropDuplicates(["venda_id"])
            .filter(col("quantidade") > 0)
            # Categoria
            .join(
                silver_dim_categoria.select("categoria_id", "sk_categoria"),
                on="categoria_id",
                how="left"
            )
            # Cliente
            .join(
                silver_dim_cliente.select("cliente_id", "sk_cliente"),
                on="cliente_id",
                how="left"
            )
            # Produto
            .join(
                silver_dim_produto.select("produto_id", "sk_produto"),
                on="produto_id",
                how="left"
            )
            # Data
            .join(
                silver_dim_data.select("data_id", "sk_data"),
                on="data_id",
                how="left"
            )
            # Localidade
            .join(
                silver_dim_localidade.select("localidade_id", "sk_localidade"),
                on="localidade_id",
                how="left"
            )
    )
    # Removendo FKs null
    silver_fato_vendas = silver_fato_vendas.filter(
        col("sk_categoria").isNotNull() & 
        col("sk_cliente").isNotNull() &
        col("sk_produto").isNotNull() &
        col("sk_data").isNotNull() &
        col("sk_localidade").isNotNull()
    )

    # Removendo id's
    silver_fato_vendas = silver_fato_vendas.drop(
        "categoria_id",
        "cliente_id",
        "produto_id",
        "data_id",
        "localidade_id"
    )

## Salvar arquivos na camada Silver

**Função de Save**

In [0]:
def save():
    silver_tables = {
        "silver_dim_categoria_produto": silver_dim_categoria,
        "silver_dim_cliente": silver_dim_cliente,
        "silver_dim_data": silver_dim_data,
        "silver_dim_localidade": silver_dim_localidade,
        "silver_dim_produto": silver_dim_produto,
        "silver_fato_vendas": silver_fato_vendas
    }

    for nome_tabela, df in silver_tables.items():
        (
            df.write
            .format("delta")
            .mode("overwrite")
            .save(f"{silver_path}/{nome_tabela}")
        )

# Main

## Limpar memória

In [0]:
gc.collect()

# **Extra: printando tabelas transformadas**

In [0]:
%skip
silver_dim_categoria_produto = spark.read.format("delta").load("/Volumes/data_ex/silver/download001/silver_dim_categoria_produto")
display(silver_dim_categoria_produto)

silver_dim_cliente = spark.read.format("delta").load("/Volumes/data_ex/silver/download001/silver_dim_cliente")
display(silver_dim_cliente)

silver_dim_data = spark.read.format("delta").load("/Volumes/data_ex/silver/download001/silver_dim_data")
display(silver_dim_data)

silver_dim_localidade = spark.read.format("delta").load("/Volumes/data_ex/silver/download001/silver_dim_localidade")
display(silver_dim_localidade)

silver_dim_produto = spark.read.format("delta").load("/Volumes/data_ex/silver/download001/silver_dim_produto")
display(silver_dim_produto)

silver_fato_vendas = spark.read.format("delta").load("/Volumes/data_ex/silver/download001/silver_fato_vendas")
display(silver_fato_vendas)

In [0]:
silver_fato_vendas = spark.read.format("delta").load("/Volumes/data_ex/silver/download001/silver_fato_vendas")
display(silver_fato_vendas)
